In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# import sys
# sys.path.append('/content/drive/MyDrive/Major_Project_Cse')

%cd "/content/drive/MyDrive/Major_Project_Cse"

In [1]:
!pip install mediapipe
import cv2
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow

mp_pose = mp.solutions.pose

# Setup the Pose function for images - independently for the images standalone processing.
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.8,model_complexity=2)

# Setup the Pose function for videos - for video processing.
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)

# Initialize mediapipe drawing class - to draw the landmarks points.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 17.6 MB/s eta 0:00:00


In [2]:
def mpPoseImg(image_pose, pose=pose_image, draw=False, display=False):
    
    annotated_image = image_pose.copy()
    
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
 
    
    results = pose.process(image_in_RGB)

    if results.pose_landmarks and draw:    
      mp_drawing.draw_landmarks(image=annotated_image,landmark_list=results.pose_landmarks,connections=mp_pose.POSE_CONNECTIONS,
                                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())


        # mp_drawing.draw_landmarks(image=original_image, landmark_list=resultant.pose_landmarks,connections=mp_pose.POSE_CONNECTIONS,
        #                           landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),thickness=3, circle_radius=3),
        #                           connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),thickness=2, circle_radius=2))

    if display:
            
            # plt.figure(figsize=[22,22])
            # plt.subplot(121);plt.imshow(image_pose[:,:,::-1]);plt.title("Input Image");plt.axis('off');
            # plt.subplot(122);plt.imshow(annotated_image[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');
            cv2_imshow(annotated_image)
            return results

    else:
        
        return annotated_image,results

In [3]:
numOfKeypoints = 34
def getKeyPointsforFrame(frame):
  img,results = mpPoseImg(frame)
  keypoints = [[None,None] for _ in range(numOfKeypoints)]
  if results.pose_landmarks == None:
    return keypoints
    
  landmarks = results.pose_landmarks
  for i in range(numOfKeypoints-2):
     mark = landmarks.landmark[i]
     keypoints[i][0]=mark.x
     keypoints[i][1]=mark.y
  #------------------------------------------------------------------------------------------------
  try:
    # MID HIP COORDINATES
      left_hip_index = 23
      right_hip_index = 24
      Mid_hip_x =  (keypoints[left_hip_index][0]+keypoints[right_hip_index][0])/2
      Mid_hip_y =(keypoints[left_hip_index][1]+keypoints[right_hip_index][1])/2
      # keypoints.append([Mid_hip_x,Mid_hip_y])
      keypoints[32][0]=Mid_hip_x
      keypoints[32][1]=Mid_hip_y
                          
  except:
      pass
      
  #------------------------------------------------------------------------------------------------
  try:
    # NECK COORDINATES
      left_shoulder_index = 11
      right_shoulder_index = 12
      Neck_x =  (keypoints[left_shoulder_index][0]+keypoints[right_shoulder_index][0])/2
      Neck_y =(keypoints[left_shoulder_index][1]+keypoints[right_shoulder_index][1])/2
      # keypoints.append([Neck_x,Neck_y])
      keypoints[33][0]=Neck_x
      keypoints[33][1]=Neck_y
                          
  except:
      pass

  #------------------------------------------------------------------------------------------------
     
  return keypoints

In [ ]:
imagePath='Images/yoga.jpg'
img = cv2.imread(imagePath)
getKeyPointsforFrame(img)

In [ ]:

import numpy as np
import pandas as pd
# import tensorflow as tf
import os,cv2

import warnings
warnings.filterwarnings("ignore")

data_path = './Videos'



labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107


train_videos=[]
train_labels=[]
train_keypoints = []
import traceback
import random

num = 1

for i,folder in enumerate(labels):
  try:
    for video in os.listdir(data_path+'/'+folder):
      print(num)
      num+=1
      video = os.path.join(data_path+'/'+folder+'/'+video)
      # Open the video file
      cap = cv2.VideoCapture(video)

      # Get the total number of frames in the video
      total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

      # Select a random starting frame number
      start_frame = random.randint(0, total_frames - 16)

      # Set the number of frames to be selected
      num_frames = 16

      # Set the frame number to start with
      cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

      # Loop through the frames and save them
      frames = []
      keypoints = []
      for j in range(num_frames):
          ret, image = cap.read()
          if ret:
            frames.append(cv2.resize(image,(112,112)))
            kk = getKeyPointsforFrame(image)
            keypoints.append(kk)

      # Release the video file
      cap.release()


      # train_videos.append(frames)
      train_keypoints.append(keypoints)
      train_labels.append(i)
      # if(num>2):
      #    break

  except Exception:
    traceback.print_exc()


In [ ]:


# create a DataFrame to store the data
data = pd.DataFrame({'keypoints': train_keypoints, 'label': train_labels})

# save the DataFrame to a CSV file
data.to_csv('train_data.csv', index=False)